In [14]:
from sentence_transformers import SentenceTransformer
sentences = ["الجزائر في حالة جيدة", "أشعر بنعاس شديد"]

model = SentenceTransformer('medmediani/Arabic-KW-Mdel', device='cpu')
embeddings = model.encode(sentences)
print(embeddings)

[[-0.73837394  0.6461413  -0.18008737 ...  0.19166851 -1.481444
  -0.37603608]
 [ 0.347957    0.26561263 -0.21292533 ...  1.2921435  -1.3724698
   0.09124265]]


In [3]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

model = SentenceTransformer('medmediani/Arabic-KW-Mdel')
index = faiss.read_index('vector_index.faiss')

def search(prompt, n=10):
    query_vector = model.encode(prompt).reshape(1, -1)
    distances, indices = index.search(query_vector, n)
    return indices

In [23]:
import pandas as pd

    df = pd.read_csv('data/data.csv')
df.head()

,id,bait_clean
0,1,خليلي لا تستعجلا أن تزودا وأن تجمعا شملي وتنتظ...
1,2,فما لبث يوما بسابق مغنم ولا سرعتي يوما بسابقة ...
2,3,وإن تنظراني اليوم أقض لبانة وتستوجبا منا علي و...
3,4,لعمرك ما نفس بجد رشيدة تؤامرني سرا لأصرم مرثدا
4,5,وإن ظهرت منه قوارص جمة وأفرع في لومي مرارا وأصعدا


In [4]:
import numpy as np
import faiss

index = faiss.read_index('armodel_index.faiss')

In [22]:
query = 'أسد قوي'
query_vector = model.encode(query).reshape(1, -1)
query_vector

array([[-0.5067792 ,  0.9514817 ,  0.17900617,  0.1520183 ,  0.36325145,
        -0.43392664, -1.5986176 ,  0.34136638, -0.3082999 , -0.21695757,
         0.6688446 ,  0.5713231 ,  0.22196633,  0.35792547, -0.11519901,
         0.13542017,  0.5319721 , -0.5138683 ,  0.3106361 , -1.1410075 ,
         0.6758803 , -0.75067574,  0.9887895 , -3.3223252 ,  0.9164412 ,
        -0.35990572, -0.1436673 ,  0.42171007, -1.2065026 , -0.13136142,
        -2.1985383 ,  0.2807588 , -0.66164505,  0.15343717, -0.7928587 ,
        -0.56848246,  0.3149493 , -0.5166376 ,  0.89588004,  0.17008567,
        -0.5800972 ,  0.8418421 ,  0.8074881 ,  0.51516235, -0.89995724,
        -0.5989208 , -0.4062454 ,  0.9444727 , -0.98504597, -0.2309865 ,
        -0.40016347, -0.03506904,  0.6178512 ,  0.03654845, -0.29970643,
        -0.16530462, -1.3867801 , -0.42192462, -0.459589  ,  0.49554172,
         1.1418581 ,  0.21142626, -0.37169567,  1.1531368 , -0.9375516 ,
        -0.05620959,  0.1711944 , -0.13978419,  1.1

In [23]:
k = 10
distances, indices = index.search(query_vector, k)

print(f'Indices of nearest neighbors: {indices}')
print(f'Distances to nearest neighbors: {distances}')

Indices of nearest neighbors: [[ 432366 1482546 1817829  916506  511237  579167 1454363 1073523 1785782
  1786847]]
Distances to nearest neighbors: [[242.27386 327.77234 329.60333 336.34314 340.59503 341.7382  350.20645
  352.4137  353.3336  353.59216]]


In [24]:
df.loc[indices.flatten()]

,البيت
432366,أسد والله لا يخشى الأسود
1482546,بفارس في حلق مدجج
1817829,وغزال يغلب الأسدا
916506,قرد يسمى فارسا يهوى ركوب الكمره
511237,غزال من الإنس
579167,ردت الأمور إلى أسد ضار
1454363,عند القوي أرانب وعلى الضعيف أسود غاب
1073523,يريد ان يفترس الانجم بالمخالب
1785782,الفرس
1786847,عصفور صغير


In [26]:
query_from_image = 'اسد'

In [28]:
def search(query):
    query_vector = model.encode(query).reshape(1, -1)
    distances, indices = index.search(query_vector, 5)
    df_prompt_result = df.loc[indices.flatten()]
    df_prompt_result.reset_index(inplace=True)
    return df_prompt_result['id'].values

search(query_from_image)

array([1825116,  612016, 1828677, 1828700, 1828527])

In [5]:
prompts = [
    'ليت الشباب يعود',
    'ثلوج الجبال',
    'المنية مفاجئة',
    'الحب'
]

f = open("output_model.txt", 'w')

for prompt in prompts:
    f.write(prompt + "\n")
    
    query_vector = model.encode(prompt).reshape(1, -1)
    distances, indices = index.search(query_vector, 50)
    df_prompt_result = df.loc[indices.flatten()]
    df_prompt_result['n'] = df_prompt_result['البيت'].apply(len)
    df_prompt_result = df_prompt_result.loc[df_prompt_result['n'] > 35]
    df_prompt_result.reset_index(inplace=True)
    
    f.write(df_prompt_result.at[0, 'البيت'] + "\n")
    f.write(df_prompt_result.at[1, 'البيت'] + "\n")
    f.write(df_prompt_result.at[2, 'البيت'] + "\n")
    
    f.write("\n")

f.close()

In [15]:
prompts = [
    'الحماسة والفخر',
    'روعة البحر',
    'الأمل والتفاؤل',
    'معاني الحياة والموت',
    'وجدانية الحب العذري',
    'التأمل في مصائر الإنسان',
    'حياة العرب قبل الإسلام',
    'العدمية والبحث عن الذات',
    'الفضيلة والأخلاق',
    'السلام والتسامح',
    'العدالة والمساواة',
    'كيف تخاف الفقر والله رازقا',
    'التشرد والهجرة والبحث عن الوطن',
    'عيناك نازلتاالقلوب',
    ' كتب الدمع بخدي عهده',
    'لا يحمل الحقد من تعلو به الرتب',
    'أتعرف رسم الدار',
    'الأصدقاء مثل النجوم',
    ' أحلك الأوقات',
    'شعر بدوي',
    ' الثقة بالله',
    ' ذكريات الطفولة',
    'المرء يأمل أن يعيش',
    'الوطن والانتماء',
    ' الحرية والنضال',
    'قم للمعلم وفه التبجيلا',
    'وعذرتهم وعرفت ذنبي',
    'إليك يا أمي',
    'فأصبح بعد الحرب',
    'إذا حكم القضاء',
    'ليت الأحزان',
    'فرحي مثل طائر',
    'نار الغضب',
    'صوت المطر ينعش روحي',
    'تنير ليالي',
    'إِن الثعالب بالضحى',
    'صبور على مر الحوادث',
    'البحث عن الحقيقة رحلة لا نهاية لها',
    'مرآة الروح',
    'كلما نسينا دروس الماضي',
    'دع الأيام تفعل ما تشاء',
    'وقفت على الديار',
    'رمال الصحراء',
    'شموخ',
    'تفاؤل',
    'نصر',
    'حمامة',
    'ساذج'
]

f = open("output_model.txt", 'w')

for prompt in prompts:
    f.write(prompt + "\n")
    
    query_vector = model.encode(prompt).reshape(1, -1)
    distances, indices = index.search(query_vector, 50)
    df_prompt_result = df.loc[indices.flatten()]
    df_prompt_result['n'] = df_prompt_result['البيت'].apply(len)
    df_prompt_result = df_prompt_result.loc[df_prompt_result['n'] > 15]
    df_prompt_result.reset_index(inplace=True)
    
    try:
        f.write(df_prompt_result.at[0, 'البيت'] + "\n")
        f.write(df_prompt_result.at[1, 'البيت'] + "\n")
        f.write(df_prompt_result.at[2, 'البيت'] + "\n")
    except:
        print('only one output for', prompt, '!')
    
    f.write("\n")

f.close()